In [40]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import OneHotEncoder
import json
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
import gzip
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import median_absolute_error

In [2]:
ruta_train = '../files/input/train_data.csv.zip'
ruta_test = "../files/input/test_data.csv.zip"

ruta_guardado = "files/models/model.pkl"
ruta_output = "files/output/metrics.json"

#ruta_datos_x_test = "files/grading/x_test.pkl"
#ruta_datos_y_test="files/grading/y_test.pkl"
#ruta_datos_x_train="files/grading/x_train.pkl"
#ruta_datos_y_train="files/grading/y_train.pkl"

In [3]:
def read(ruta): 
  
    df = pd.read_csv(ruta, compression='zip')
 
    return df


In [4]:
df_train = read(ruta_train)
df_test = read(ruta_test)

In [5]:
def clear(df):
    
    df = df.copy()

    df['Age']=2021-df['Year']

    df = df.drop(columns=['Year', 'Car_Name'])
    df['Owner'] = df['Owner'].astype(str).astype('category')
    # Convertir 'Age' a tipo categórico
    # df['Age'] = df['Age'].astype('category')
    # df['Owner'] = df['Owner'].astype('category')
    # df['Fuel_Type'] = df['Fuel_Type'].astype('category')
    # df['Selling_type'] = df['Selling_type'].astype('category')
    # df['Transmission'] = df['Transmission'].astype('category')
    
    return df

In [6]:
df_train = clear(df_train)
df_test=clear(df_test)


In [7]:
print(df_train[['Owner']].value_counts())
print(df_train['Owner'].cat.categories)

Owner
0        203
1          7
3          1
Name: count, dtype: int64
Index(['0', '1', '3'], dtype='object')


In [8]:
def dividr_datos(df):
    df = df.copy()
    X=df.drop(columns=["Present_Price"])
    y=df["Present_Price"]
    return  X,y

In [9]:
x_train, y_train=dividr_datos(df_train)
x_test, y_test= dividr_datos(df_test)


In [10]:
# categorical_columns = x_train.select_dtypes(include=['object', 'category']).columns.tolist()
# numerical_features = x_train.columns.drop(categorical_columns).tolist()
# print(numerical_features)
# categorical_columns


In [ ]:
categorical_columns=['Fuel_Type', 'Selling_type', 'Transmission']
numerical_features = ['Selling_Price', 'Driven_kms', 'Age','Owner']

# Preprocesamiento
preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_columns),
            ("scales", MinMaxScaler(), numerical_features)
            
        ],
        remainder="passthrough"
        
    )
# Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', SelectKBest(score_func=f_regression)),
    ('regression',  LinearRegression())
])

# # Definir la búsqueda de hiperparámetros
param_grid = {
    'feature_selection__k':range(1, 12),   # Número de características a seleccionar
        }


# Crear el objeto de búsqueda
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=10, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1,
    refit=True
)
modelo=grid_search.fit(x_train, y_train)

# Resultados
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)
# Evaluar el modelo en el conjunto de prueba
y_pred = grid_search.best_estimator_.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mad= mean_absolute_error(y_test, y_pred)

print("Error cuadrático medio en prueba (MSE):", mse)
print("Coeficiente de determinación en prueba (R²):", r2)
print("Error cuadrático medio en prueba (MSE):", mse)
print("Coeficiente de determinación en prueba (mad):", mad)

# # Evaluar el modelo en el conjunto de prueba
# y_pred = grid_search.best_estimator_.predict(x_test)
# test_score = balanced_accuracy_score(y_test, y_pred)
# print("Precisión balanceada en el conjunto de prueba:", test_score)

Mejores parámetros encontrados: {'feature_selection__k': 11}
Mejor puntuación de validación cruzada: -8.300109997431653
Error cuadrático medio en prueba (MSE): 32.566672753866236
Coeficiente de determinación en prueba (R²): 0.7325716754123308
Error cuadrático medio en prueba (MSE): 32.566672753866236
Coeficiente de determinación en prueba (mad): 2.4736199144163753


In [44]:
print("Columnas en el conjunto de datos de entrenamiento:", x_train.columns)
print("Columnas en el conjunto de datos de prueba:", x_test.columns)

Columnas en el conjunto de datos de entrenamiento: Index(['Selling_Price', 'Driven_kms', 'Fuel_Type', 'Selling_type',
       'Transmission', 'Owner', 'Age'],
      dtype='object')
Columnas en el conjunto de datos de prueba: Index(['Selling_Price', 'Driven_kms', 'Fuel_Type', 'Selling_type',
       'Transmission', 'Owner', 'Age'],
      dtype='object')


In [45]:
ruta_guardado = "../files/models/model.pkl.gz"

# Asegúrate de que el directorio existe
ruta_guardado_carpeta = os.path.dirname(ruta_guardado)
os.makedirs(ruta_guardado_carpeta, exist_ok=True)

# Guarda el modelo
with gzip.open(ruta_guardado, 'wb') as f:
    pickle.dump(grid_search, f)

In [46]:
def calculate_and_save_metrics(model, X_train, X_test, y_train, y_test):
    # Hacer predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcular métricas para el conjunto de entrenamiento
    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'r2': r2_score(y_train, y_train_pred),
        'mse': mean_squared_error(y_train, y_train_pred),
        'mad': median_absolute_error(y_train, y_train_pred),
        }

    # Calcular métricas para el conjunto de prueba
    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'r2': r2_score(y_test, y_test_pred),
        'mse': mean_squared_error(y_test, y_test_pred),
        'mad': median_absolute_error(y_test, y_test_pred),#####ERROR EN METRICA UTILIZADA#####
        
    }

    # Crear carpeta si no existe
    output_dir = '../files/output'
    os.makedirs(output_dir, exist_ok=True)

    # Guardar las métricas en un archivo JSON
    output_path = os.path.join(output_dir, 'metrics.json')
    with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
        f.write(json.dumps(metrics_train) + '\n')
        f.write(json.dumps(metrics_test) + '\n')


In [48]:
def main(modelo, X_train, X_test, y_train, y_test):
    # Crear el directorio de salida si no existe
    import os
    os.makedirs('../files/output', exist_ok=True)

    # Calcular y guardar las métricas
    calculate_and_save_metrics(modelo, X_train, X_test, y_train, y_test)

  
# Ejemplo de uso:
main(modelo, x_train, x_test, y_train, y_test)

In [16]:
def _load_model():
    ruta_guardado = "../files/models/model.pkl.gz"
    with gzip.open(ruta_guardado, 'rb') as f:
        model=joblib.load(f)
        return model


In [17]:
model = _load_model()
print(type(model))  # Asegúrate de que sea <class 'GridSearchCV'>


<class 'sklearn.model_selection._search.GridSearchCV'>


In [18]:
def _load_model():
    ruta_guardado = "../files/models/model.pkl.gz"
    """Generic test to load a model"""
    with gzip.open(ruta_guardado, "rb") as file:
        model = pickle.load(file)
    return model


In [19]:
modelo=_load_model()
print(type(modelo))
modelo

<class 'sklearn.model_selection._search.GridSearchCV'>


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission']),
                                                                        ('scales',
                                                                         MinMaxScaler(),
                                                                         ['Selling_Price',
                                                                          'Driven_kms',
                                                                          'Owner',
                                                                          'Age'])])),
                                       ('feature_selection',
                                        SelectKBest(score_func=<function f_regression at 0x0000025DBC4AEB90>)),
                                       ('regression', LinearRegression())]),
             n_jobs=-1, param_grid={'feature_selection__k': range(1, 12)},
             scoring='neg_mean_squared_error')